# Imports

In [100]:
from numpy.random import seed
seed(888)
from tensorflow import random
#random.set_seed(404)

import os
import numpy as np
import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

from time import strftime
from PIL import Image

AttributeError: module 'tensorflow.compat.v1' has no attribute 'set_seed'

# Constants

In [52]:
X_TRAIN_FILE_PATH = "C:/Users/BediZ/DS-ML/Jupyter Notebook - ML Projects/resources/MNIST/digit_xtrain.csv"
Y_TRAIN_FILE_PATH = "C:/Users/BediZ/DS-ML/Jupyter Notebook - ML Projects/resources/MNIST/digit_ytrain.csv"
X_TEST_FILE_PATH = "C:/Users/BediZ/DS-ML/Jupyter Notebook - ML Projects/resources/MNIST/digit_xtest.csv"
Y_TEST_FILE_PATH = "C:/Users/BediZ/DS-ML/Jupyter Notebook - ML Projects/resources/MNIST/digit_ytest.csv"
IMAGE_TEST_PATH = "C:/Users/BediZ/DS-ML/Jupyter Notebook - ML Projects/resources/MNIST/test_img.png"

LOGGING_PATH = 'tensorboard_mnist_digit_logs/'

NUMBER_OF_CLASSES = 10
VALIDATION_SIZE = 10000

IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
NUMBER_OF_CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH * IMAGE_HEIGHT * NUMBER_OF_CHANNELS

# Get the Data

In [53]:
%%time

y_train_all = np.loadtxt(Y_TRAIN_FILE_PATH, delimiter=',', dtype=int)
y_test = np.loadtxt(Y_TEST_FILE_PATH, delimiter=',', dtype=int)

x_train_all = np.loadtxt(X_TRAIN_FILE_PATH, delimiter=',', dtype=int)
x_test = np.loadtxt(X_TEST_FILE_PATH, delimiter=',', dtype=int)

Wall time: 26.7 s


# Explore the data

In [54]:
x_train_all.shape

(60000, 784)

In [55]:
x_train_all[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [56]:
y_train_all.shape

(60000,)

In [57]:
y_train_all[:5]

array([5, 0, 4, 1, 9])

In [58]:
x_test.shape

(10000, 784)

# Data PreProcessing

In [59]:
# Re-scale
x_train_all, x_test = x_train_all / 255.0, x_test / 255.0

#### Converte target to one-hot encoding

In [60]:
values= y_train_all[:5]
np.eye(NUMBER_OF_CLASSES)[values]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [61]:
y_train_all = np.eye(NUMBER_OF_CLASSES)[y_train_all]
y_train_all.shape

(60000, 10)

In [62]:
y_test = np.eye(NUMBER_OF_CLASSES)[y_test]
y_test.shape

(10000, 10)

### Create validation dataset from training data

In [63]:
x_validation = x_train_all[:VALIDATION_SIZE]
y_validation = y_train_all[:VALIDATION_SIZE]

x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

x_train.shape

(50000, 784)

In [64]:
x_validation.shape

(10000, 784)

# Setup TensorFlow Graph

In [65]:
X = tf.placeholder(tf.float32, shape=[None, TOTAL_INPUTS], name='X')
Y = tf.placeholder(tf.float32, shape=[None, NUMBER_OF_CLASSES], name='labelsY')

### Neural Network Architecture
#### Hyperparameters - detirmen by me - how long to train model - number of epochs - learning rate - number of layers - number of nodes ...

In [66]:
number_of_epochs = 50
learning_rate = 1e-4 #0.0001 - 1st test
learning_rate = 1e-3 #0.001 - 2nd test run
number_of_neurons_hidden1 = 512
number_of_neurons_hidden2 = 64

# Code for 1st part of module

In [67]:
# with tf.name_scope('hidden_output_layer_1'):
#     # 1st layer of neurons
#     initial_weights1= tf.truncated_normal(shape = [TOTAL_INPUTS, number_of_neurons_hidden1], stddev = 0.1, seed = 42)
#     weights1 = tf.Variable(initial_value = initial_weights1, name='Weight_1')
#     # where the network is learning
#     initial_bias1 = tf.constant(value=0.0, shape=[number_of_neurons_hidden1])
#     bias1 = tf.Variable(initial_value = initial_bias1, name='Bias_1')
#     in_layer1 = tf.matmul(X, weights1) + bias1
#     out_layer1 = tf.nn.relu(in_layer1)
    

In [68]:
# with tf.name_scope('hidden_output_layer_2'):
#     # 2nd layer of neural
#     initial_weights2= tf.truncated_normal(shape = [number_of_neurons_hidden1, number_of_neurons_hidden2], 
#                                           stddev = 0.1, 
#                                           seed = 42)
#     weights2 = tf.Variable(initial_value = initial_weights2, name='Weight_2')
#     # where the network is learning
#     initial_bias2 = tf.constant(value=0.0, shape=[number_of_neurons_hidden2])
#     bias2 = tf.Variable(initial_value = initial_bias2, name='Bieas_2')

#     in_layer2 = tf.matmul(out_layer1, weights2) + bias2
#     out_layer2 = tf.nn.relu(in_layer2)

In [69]:
# with tf.name_scope('output_layer'):

#     # out - last layer of neural -> finis with the size od classes - softmax for output
#     initial_weights3= tf.truncated_normal(shape = [number_of_neurons_hidden2, NUMBER_OF_CLASSES], 
#                                           stddev = 0.1, 
#                                           seed = 42)
#     weights3 = tf.Variable(initial_value = initial_weights3, name='Weight_3')
#     # where the network is learning
#     initial_bias3 = tf.constant(value=0.0, shape=[NUMBER_OF_CLASSES])
#     bias3 = tf.Variable(initial_value = initial_bias3, name='Bias_3')

#     in_layer3 = tf.matmul(out_layer2, weights3) + bias3
#     output_out_layer3 = tf.nn.softmax(in_layer3)

In [70]:
def setup_layer(initial_previous_out_layer, weight_dimensions, bias_dimensions, name):
    with tf.name_scope(name):

        # out - last layer of neural -> finis with the size od classes - softmax for output
        initial_weights= tf.truncated_normal(
            shape = weight_dimensions,
            stddev = 0.1, 
            seed = 42)
        weights = tf.Variable(initial_value = initial_weights, name='Weight')
        # where the network is learning
        initial_bias = tf.constant(value=0.0, shape=bias_dimensions)
        bias = tf.Variable(initial_value = initial_bias, name='Bias')

        in_layer = tf.matmul(initial_previous_out_layer, weights) + bias
        
        if name == 'out':        
            output_out_layer = tf.nn.softmax(in_layer)
        else:            
            output_out_layer = tf.nn.relu(in_layer)
            
            
        tf.summary.histogram('weights', weights)
        tf.summary.histogram('bias', bias)    
        
        return output_out_layer

In [71]:
# Model without dropout
# out_layer1 = setup_layer(
#     X, 
#     weight_dimensions=[TOTAL_INPUTS, number_of_neurons_hidden1], 
#     bias_dimensions=[number_of_neurons_hidden1], 
#     name='out_layer1')
# out_layer2 = setup_layer(
#     out_layer1, 
#     weight_dimensions=[number_of_neurons_hidden1, number_of_neurons_hidden2], 
#     bias_dimensions=[number_of_neurons_hidden2], 
#     name='out_layer2')
# output_out_layer3 = setup_layer(
#     out_layer2, 
#     weight_dimensions=[number_of_neurons_hidden2, NUMBER_OF_CLASSES], 
#     bias_dimensions=[NUMBER_OF_CLASSES], 
#     name='output_out_layer3')

# model_name= f'{number_of_neurons_hidden1}-{number_of_neurons_hidden2} LR{learning_rate} E{number_of_epochs}'

In [72]:
out_layer1 = setup_layer(
    X, 
    weight_dimensions=[TOTAL_INPUTS, number_of_neurons_hidden1], 
    bias_dimensions=[number_of_neurons_hidden1], 
    name='out_layer1')

#dropout layer
out_layer_drop = tf.nn.dropout(
    out_layer1, 
    keep_prob=0.8, 
    name='out_layer_drop')

out_layer2 = setup_layer(
    out_layer_drop, 
    weight_dimensions=[number_of_neurons_hidden1, number_of_neurons_hidden2], 
    bias_dimensions=[number_of_neurons_hidden2], 
    name='out_layer2')
output_out_layer3 = setup_layer(
    out_layer2, 
    weight_dimensions=[number_of_neurons_hidden2, NUMBER_OF_CLASSES], 
    bias_dimensions=[NUMBER_OF_CLASSES], 
    name='output_out_layer3')

model_name= f'{number_of_neurons_hidden1}-DO-{number_of_neurons_hidden2} LR{learning_rate} E{number_of_epochs}'

# Tensorboard Setup

In [73]:
# Folder for Tensorboard

folder_name = f'{model_name}_at_{strftime("%H_%M_%S")}'
directory_tensorborad = os.path.join(LOGGING_PATH, folder_name)

try:
    os.makedirs(directory_tensorborad)
except OSError as exception:
        print('Folder NOT created at location:', directory_tensorborad)
        print(exception.strerror)
else:
    print('Folder successfuly created at location:', directory_tensorborad)

Folder successfuly created at location: tensorboard_mnist_digit_logs/512-DO-64 LR0.001 E50_at_13_31_20


# Loss, Optimization and Metrics

### Defining Loss Function

In [74]:
with tf.name_scope('loss_calculation'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output_out_layer3))

### Defining Optimizer

In [75]:
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_step = optimizer.minimize(loss)

### Accuracy Metric

In [76]:
with tf.name_scope('accuracy_calculation'):
    correct_prediction = tf.equal(tf.argmax(output_out_layer3, axis=1), tf.argmax(Y, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [77]:
with tf.name_scope('performance'):
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('loss-cost', loss)

#### Check Input Images in Tensorboard

In [78]:
with tf.name_scope('show_image'):
    x_image = tf.reshape(X, [-1, 28, 28, 1])
    tf.summary.image('image_input', x_image, max_outputs=4)

## Run Session

In [79]:
session = tf.Session()

#### Read Setup Filewriter and Merge Summaries

In [80]:
merged_summary = tf.summary.merge_all()

file_writer = tf.summary.FileWriter(directory_tensorborad + '/train')
file_writer.add_graph(session.graph)

In [81]:
validation_writer = tf.summary.FileWriter(directory_tensorborad + '/validation')

# Initialize all the variables

In [82]:
initialized_variables = tf.global_variables_initializer()
session.run(initialized_variables)

In [83]:
#weights1.eval(session)

In [84]:
#bias3.eval(session)

### Batching the Data

In [85]:
size_of_batch = 1000

In [86]:
number_of_examples = y_train.shape[0]
number_of_iteratons = int(number_of_examples / size_of_batch)

index_in_epoch = 0

In [87]:
def next_batch(batch_size, data_set, labels):
    global number_of_examples
    global index_in_epoch
    
    start_point = index_in_epoch
    index_in_epoch = index_in_epoch + batch_size
    
    if index_in_epoch > number_of_examples:
        start = 0
        index_in_epoch = batch_size
    
    end_point = index_in_epoch
    
    return data_set[start_point:end_point], labels[start_point:end_point]

### Training Loop

In [88]:
for epoch in range(number_of_epochs):
    
    # ========== Training Dataset ========== #
    for i in range(number_of_iteratons):
        batch_x, batch_y = next_batch(batch_size=size_of_batch, data_set = x_train, labels=y_train)
        #print('x', batch_x.shape, 'y', batch_y.shape)
        feed_dictionary = {X:batch_x, Y:batch_y}
        session.run(train_step, feed_dict=feed_dictionary)
        
    s, batch_accuracy = session.run(fetches=[merged_summary, accuracy], feed_dict=feed_dictionary)
    
    file_writer.add_summary(s, epoch)
    
    print(f'Epoch number {epoch} \t- Training Accuracy = {batch_accuracy}')
    # ========== Validation ========== #
    summery = session.run(fetches=merged_summary, feed_dict={X:x_validation, Y:y_validation})
    validation_writer.add_summary(summery, epoch)
print('Done training!') 
print('In anaconda prompt type: \n tensorboard --logdir=\"C:\\Users\\BediZ\\DS-ML\\Jupyter Notebook - ML Projects\\05 Neural Nets - Retrainer Image Classification\\tensorboard_mnist_digit_logs\" \n and you will get a link to TensorBoard linke: http://localhost:6006/')

Epoch number 0 	- Training Accuracy = 0.9300000071525574
Epoch number 1 	- Training Accuracy = 0.9570000171661377
Epoch number 2 	- Training Accuracy = 0.9700000286102295
Epoch number 3 	- Training Accuracy = 0.9739999771118164
Epoch number 4 	- Training Accuracy = 0.9810000061988831
Epoch number 5 	- Training Accuracy = 0.9850000143051147
Epoch number 6 	- Training Accuracy = 0.9850000143051147
Epoch number 7 	- Training Accuracy = 0.9890000224113464
Epoch number 8 	- Training Accuracy = 0.9879999756813049
Epoch number 9 	- Training Accuracy = 0.9890000224113464
Epoch number 10 	- Training Accuracy = 0.9890000224113464
Epoch number 11 	- Training Accuracy = 0.9909999966621399
Epoch number 12 	- Training Accuracy = 0.9929999709129333
Epoch number 13 	- Training Accuracy = 0.9919999837875366
Epoch number 14 	- Training Accuracy = 0.9950000047683716
Epoch number 15 	- Training Accuracy = 0.9940000176429749
Epoch number 16 	- Training Accuracy = 0.9929999709129333
Epoch number 17 	- Train

# Make a Prediction

In [89]:
image_test = Image.open("C:/Users/BediZ/DS-ML/Jupyter Notebook - ML Projects/resources/MNIST/test_img.png")
image_test

In [90]:
image_test_black_white = image_test.convert(mode='L')
image_array = np.invert(image_test_black_white)
image_array.shape

(28, 28)

In [91]:
image_flat = image_array.ravel()
image_flat.shape

(784,)

In [92]:
image_test_prediction = session.run(fetches=tf.argmax(output_out_layer3, axis=1), feed_dict={X:[image_flat]})

In [93]:
print(f'Prediction for test image _2_ is: {image_test_prediction}')

Prediction for test image _2_ is: [2]


# Testing and Evaluating

In [94]:
test_accuracy = session.run(fetches=accuracy, feed_dict={X:x_test, Y:y_test})
print(f'Prediction accuracy for test test set is: {test_accuracy:0.2%}')

Prediction accuracy for test test set is: 97.84%


# Reset the Next Run

In [95]:
file_writer.close()
validation_writer.close()
session.close()
tf.reset_default_graph()